In [ ]:
#!apt install cmake

In [3]:
!git clone https://github.com/cloner174/mr.DGA.git

Cloning into 'mr.DGA'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 254 (delta 10), reused 20 (delta 5), pack-reused 227
Receiving objects: 100% (254/254), 234.32 MiB | 19.61 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Updating files: 100% (57/57), done.


In [2]:
from google.colab import drive
drive.mount('/content/new/')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%cp '/content/new/MyDrive/Colab Notebooks/data-pix.csv' '/content/mr.DGA/data/Faze1/input'

In [ ]:
%mv '/content/mr.DGA/data/Faze1/input/data-pix.csv' '/content/mr.DGA/data/Faze1/input/dataset.csv'

In [ ]:
import pandas as pd

In [ ]:
data_path = '/content/mr.DGA/data/Faze1/input/dataset.csv'

In [ ]:
# Load data

data = pd.read_csv(data_path, compression='zip')

In [ ]:
%pip install imutils opencv-python dlib opencv-python-headless pillow matplotlib face_recognition mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=171be800b484787923d08a0fdf5bdaa8cdd258da66ca9883bea3c57d2aebb8fd
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import cv2
import face_recognition
import numpy as np
import os

In [ ]:
def strings_with_number_inside( data,
                                    column_range_starts : int = 0,
                                    column_range_ends : int = None,
                                    seperator = None):
        start = column_range_starts
        end = int( data.shape[1] ) if column_range_ends is None else column_range_ends
        new_data = {}
        if isinstance( data , pd.DataFrame ) :
            for i in range( data.shape[0] ) :
                temp_new_data = []
                for j in range( start, end ) :
                    cells_real_values = np.array(data.iloc[i, j].split(sep = seperator), dtype='uint8')
                    temp_new_data.append(cells_real_values)
                new_data[i] = temp_new_data

            return new_data
        else:
            if isinstance( data , np.array ) :
                pass
            else:
                raise TypeError( " data just could be  PandasDataFrame  or  NumpyArray  ! ")

In [ ]:
# path to save images
face_directory = '/content/data/Faze1/output/face_directory/'
non_face_directory = '/content/data/Faze1/output/non_face/'

In [ ]:
data_pix = strings_with_number_inside(data, column_range_starts = 1)

In [ ]:
# each row in corresponds to an image
image_size = (48, 48)  # dimensions of the images

In [ ]:
os.makedirs(face_directory, exist_ok=True)
os.makedirs(non_face_directory, exist_ok=True)

for index, row in data_pix.items():
    image_array = np.array(row, dtype=np.uint8).reshape(image_size[0], image_size[1])

    image_rgb = np.stack((image_array,) * 3, axis=-1)

    face_locations = face_recognition.face_locations(image_rgb, model='cnn')

    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image_array[top:bottom, left:right]
        cv2.imwrite(os.path.join(face_directory, f'face_{index}.jpg'), face_image)

    if len(face_locations) == 0:
        cv2.imwrite(os.path.join(non_face_directory, f'non_face_{index}.jpg'), image_array)

print("face cropping and saving completed.")


face cropping and saving completed.


In [ ]:
in_face_folder = os.listdir(face_directory)
non_face_folder = os.listdir(non_face_directory)

In [ ]:
len(in_face_folder), len(non_face_folder)

(31519, 4368)

In [ ]:
filter

In [ ]:
!zip -r '/content/face_directory.zip' '/content/data/face_directory/'
!zip -r '/content/non_face.zip' '/content/data/non_face'

In [ ]:
face_cascade = cv2.CascadeClassifier('/content/mr.DGA/mr.DGA/models/haarcascade_frontalface_default.xml')

In [ ]:

def detect_faces(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return len(faces)


In [ ]:
invali_trues = []
for any_ in in_face_folder :
  temp_path = os.path.join(face_directory, any_)
  temp_len = detect_faces(temp_path)
  if temp_len == 0 :
    invali_trues.append(any_)
  else:
    continue

In [ ]:
len(invali_trues)

29289

In [ ]:
data_pix = strings_with_number_inside(data, column_range_starts = 1)

In [ ]:
#%pip install face_recognition

In [ ]:
import dlib
import cv2
import numpy as np

predictor_path = "/content/mr.DGA/models/shape_predictor_68_face_landmarks.dat"
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor(predictor_path)

image = cv2.imread("path/to/image.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

faces = face_detector(gray)

for face in faces:
    landmarks = landmark_predictor(gray, face)
    landmarks_points = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))
        cv2.circle(image, (x, y), 2, (255, 0, 0), -1)

    rect = (0, 0, image.shape[1], image.shape[0])
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks_points)
    triangles = subdiv.getTriangleList()

    for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        cv2.line(image, pt1, pt2, (0, 255, 0), 1)
        cv2.line(image, pt2, pt3, (0, 255, 0), 1)
        cv2.line(image, pt3, pt1, (0, 255, 0), 1)

cv2.imshow('Output', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:

def save(data_row, img_dir, img_size=(48, 48)):

        pixels = np.array(data_row[' pixels'].split(), dtype='uint8')
        try:
            image = pixels.reshape(img_size)
        except ValueError:
            print(f"Error reshaping image: {img_size} may not be the right dimensions.")
            return False
        img = Image.fromarray(image, 'L')  # 'L' for grayscale
        file_path = os.path.join(img_dir, f"{data_row.name}.png")
        img.save(file_path)
        return True

In [ ]:
!git clone https://github.com/italojs/facial-landmarks-recognition.git

In [ ]:
from main import Images, Utilities

In [ ]:
# Base directory to store CSVs, grouped by emotion labels
base_csv_dir = 'data/Faze1/output/CSVs'
base_image_dir = 'data/Faze1/output/Pics'
labels = data['emotion'].unique()

In [ ]:
# Create directories for each label
directories = Utilities.create_directories(base_image_dir, labels)

In [ ]:
# Process each image in the dataframe
for index, row in data.iterrows():
    if not Images.save(row, directories[row['emotion']]):
        print(f"Failed to process and save image at index {index}")

In [ ]:
# Create directories for each label
directories = Utilities.create_directories(base_csv_dir, labels)

In [ ]:
for label in labels:

    subset = data[data['emotion'] == label]
    file_path = os.path.join(directories[label], f'data_label_{label}.csv')
    subset.to_csv(file_path, index=False)
    print(f"Data for label {label} saved to {file_path}")


In [ ]:
#example
test = pd.read_csv('data/Faze1/output/CSVs/6/data_label_6.csv')
test.head(), test.tail()

In [ ]:
%pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 30.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [ ]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

image = cv2.imread('image.jpg')

rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:

    results = face_mesh.process(rgb_image)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                landmark_x = landmark.x
                landmark_y = landmark.y

                print(f"Landmark ({landmark.landmark_type}) coordinates: ({landmark_x}, {landmark_y})")




In [ ]:
import os
import dlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from PIL import Image


image_directory = ''
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def plot_delaunay(points, delaunay, ax):
    for simplex in delaunay.simplices:
        ax.plot(points[simplex, 0], points[simplex, 1], 'k-')

def detect_faces_and_triangulate(image_path):
    img = Image.open(image_path)
    img = np.array(img)

    ax = plt.gca()
    ax.imshow(img)

    detections = detector(img, 1)
    for k, d in enumerate(detections):
        shape = predictor(img, d)
        points = np.array([[p.x, p.y] for p in shape.parts()])
        ax.scatter(points[:, 0], points[:, 1], c='r', s=10)
        delaunay = Delaunay(points)
        plot_delaunay(points, delaunay, ax)

    plt.axis('off')
    plt.show()

for image_file in os.listdir(image_directory):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_directory, image_file)
        detect_faces_and_triangulate(image_path)


In [ ]:
import os
import dlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from PIL import Image

image_directory = ''
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def plot_delaunay(points, delaunay, ax):
    for simplex in delaunay.simplices:
        ax.plot(points[simplex, 0], points[simplex, 1], 'k-')

def get_delaunay_for_part(points, part_indices):
    part_points = points[part_indices]
    return Delaunay(part_points)

def detect_faces_and_triangulate_parts(image_path):
    img = Image.open(image_path)
    img = np.array(img)

    ax = plt.gca()
    ax.imshow(img)

    detections = detector(img, 1)
    for k, d in enumerate(detections):
        shape = predictor(img, d)
        points = np.array([[p.x, p.y] for p in shape.parts()])
        ax.scatter(points[:, 0], points[:, 1], c='r', s=10)

        eye_indices = np.arange(36, 48)
        lip_indices = np.arange(48, 68)

        eye_delaunay = get_delaunay_for_part(points, eye_indices)
        lip_delaunay = get_delaunay_for_part(points, lip_indices)

        plot_delaunay(points, eye_delaunay, ax)
        plot_delaunay(points, lip_delaunay, ax)

    plt.axis('off')
    plt.show()

for image_file in os.listdir(image_directory):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_directory, image_file)
        detect_faces_and_triangulate_parts(image_path)
